# Image Generation

## Text to image generation

In [2]:
import base64
import io
import json
from typing import Dict, Any
from botocore.config import Config

import boto3
from PIL import Image

def generate_image(model_id: str, body: str) -> bytes:
    bedrock = boto3.client(
        service_name='bedrock-runtime',
        config=Config(read_timeout=300)
    )

    response = bedrock.invoke_model(
        body=body,
        modelId=model_id,
    )
    
    response_body = json.loads(response.get("body").read())
    base64_image = response_body.get("images")[0]
    base64_bytes = base64_image.encode('ascii')
    image_bytes = base64.b64decode(base64_bytes)

    return image_bytes

model_id = 'amazon.nova-canvas-v1:0'
prompt = "Korean tteokbokki - spicy rice cakes in bright red sauce garnished with green onions."

body = json.dumps({
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {
        "text": prompt
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "height": 512,
        "width": 512,
        "cfgScale": 8.0,
        "seed": 0
    }
})

image_bytes = generate_image(model_id=model_id, body=body)
image = Image.open(io.BytesIO(image_bytes))
image.save('food_gen.png',"PNG")

In [3]:
image.save('food_gen.png',"PNG")

## Inpainting

In [4]:
# Convert saved image to base64 for inpainting
with open("food_gen.png", "rb") as image_file:
    input_image = base64.b64encode(image_file.read()).decode('utf8')

# Inpainting to remove green onions
inpaint_body = json.dumps({
    "taskType": "INPAINTING",
    "inPaintingParams": {
        "text": "Korean tteokbokki - spicy rice cakes in bright red sauce",
        "negativeText": "green onions, garnish",
        "image": input_image,
        "maskPrompt": "green onions"
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "height": 512,
        "width": 512,
        "cfgScale": 8.0
    }
})

# Generate and save the inpainted image
inpainted_bytes = generate_image(model_id=model_id, body=inpaint_body)
inpainted_image = Image.open(io.BytesIO(inpainted_bytes))
inpainted_image.save('food_gen_no_onions.png', "PNG")

## Bad inpainting example

In [5]:
# Convert saved image to base64 for inpainting
with open("food_gen.png", "rb") as image_file:
    input_image = base64.b64encode(image_file.read()).decode('utf8')

# Inpainting to remove green onions with negative words (not working)
inpaint_body = json.dumps({
    "taskType": "INPAINTING",
    "inPaintingParams": {
        "text": "Korean tteokbokki - spicy rice cakes in bright red sauce without green onions",
        # "negativeText": "green onions, garnish",
        "image": input_image,
        "maskPrompt": "green onions"
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "height": 512,
        "width": 512,
        "cfgScale": 8.0
    }
})

# Generate and save the inpainted image
inpainted_bytes = generate_image(model_id=model_id, body=inpaint_body)
inpainted_image = Image.open(io.BytesIO(inpainted_bytes))
inpainted_image.save('food_gen_no_onions_negative_word.png', "PNG")

## Image Variation

In [7]:
# Convert saved image to base64 for variation
with open("food_gen.png", "rb") as image_file:
    input_image = base64.b64encode(image_file.read()).decode('utf8')
    
# Generate cream sauce variation
variation_body = json.dumps({
    "taskType": "IMAGE_VARIATION",
    "imageVariationParams": {
        "text": "Korean tteokbokki in creamy white sauce, rich and creamy texture",
        "negativeText": "red sauce, spicy sauce, gochujang",
        "images": [input_image],
        "similarityStrength": 0.5
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "height": 512,
        "width": 512,
        "cfgScale": 8.0
    }
})

# Generate and save the variation image
variation_bytes = generate_image(model_id=model_id, body=variation_body)
variation_image = Image.open(io.BytesIO(variation_bytes))
variation_image.save('food_gen_cream_sauce.png', "PNG")

In [8]:
import json
import boto3

def generate_video(text: str, duration: int = 6) -> dict:
    bedrock_runtime = boto3.client("bedrock-runtime")
    
    model_input = {
        "taskType": "TEXT_VIDEO",
        "textToVideoParams": {
            "text": text
        },
        "videoGenerationConfig": {
            "durationSeconds": duration,
            "fps": 24,
            "dimension": "1280x720",
            "seed": 0
        }
    }

    response = bedrock_runtime.start_async_invoke(
        modelId="amazon.nova-reel-v1:0",
        modelInput=model_input,
        outputDataConfig={
            "s3OutputDataConfig": {
                "s3Uri": "s3://my-nova-videos-korean-foods"
            }
        }
    )
    
    return response

# Generate video
text_prompt = "steaming hot tteokbokki with rich gochujang sauce and chopped green onions on top"
result = generate_video(text=text_prompt)
print(json.dumps(result, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "348d16d1-8c7f-48f9-9b56-f85cf34556fc",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 22 Jan 2025 01:29:31 GMT",
      "content-type": "application/json",
      "content-length": "84",
      "connection": "keep-alive",
      "x-amzn-requestid": "348d16d1-8c7f-48f9-9b56-f85cf34556fc"
    },
    "RetryAttempts": 0
  },
  "invocationArn": "arn:aws:bedrock:us-east-1:418272795925:async-invoke/a34g3i4i3g9c"
}
